In [1]:
import perform_imputation_by_sample_matching as pism
import util
import json
import os
import sys

import numpy as np
import pandas as pd

import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr

import tsinfer
import tskit
import tszip

sys.path.append("../../tsimpute/src/")


### Split into reference panel and target cohorts

In [2]:
data_dir = "../data/trees/"
ts_hc_bi_file = data_dir + "hgdp_tgp_sgdp_chr20_p.dated.hc_bi.trees.tsz"
ts = tszip.decompress(ts_hc_bi_file)


In [3]:
ref_size = 700
target_size = ts.num_individuals - ref_size

print(f"Reference panel size: {ref_size}")
print(f"Target cohort size: {target_size}")


Reference panel size: 700
Target cohort size: 176


In [4]:
np.random.seed(1234)

all_inds = np.arange(ts.num_individuals)
ref_inds = np.sort(np.random.choice(all_inds, ref_size, replace=False))
target_inds = np.array(list(set(all_inds) - set(ref_inds)))

print(f"Reference panel size: {len(ref_inds)}")
print(f"Target cohort size: {len(target_inds)}")

assert len(ref_inds) == ref_size
assert len(target_inds) == target_size


Reference panel size: 700
Target cohort size: 176


In [5]:
all_nodes = np.arange(ts.num_nodes)
ref_nodes = all_nodes[np.isin(ts.nodes_individual, ref_inds)]
target_nodes = all_nodes[np.isin(ts.nodes_individual, target_inds)]


In [6]:
ref_ts = ts.simplify(
    samples=ref_nodes,
    filter_populations=True,
    filter_individuals=True,
    filter_nodes=True,
    filter_sites=True,
)


In [7]:
target_ts = ts.simplify(
    samples=target_nodes,
    filter_populations=True,
    filter_individuals=True,
    filter_nodes=True,
    filter_sites=True,
)


In [8]:
ref_ts


In [9]:
target_ts


In [10]:
out_dir = "../data/trees/"
ref_ts_file = out_dir + "ref.tsz"
target_ts_file = out_dir + "target.tsz"
tszip.compress(ref_ts, ref_ts_file)
tszip.compress(target_ts, target_ts_file)
